<a href="https://colab.research.google.com/github/Israraza/Deep-Learning/blob/main/Image_Classification_using_Transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!unzip '/content/drive/My Drive/Image-Classification-Transfer-Learning-master (1).zip' -d /content/dataset

Archive:  /content/drive/My Drive/Image-Classification-Transfer-Learning-master (1).zip
2cb7ceca15a9e7a7ae7ece9e781bb1383e5bd58d
   creating: /content/dataset/Image-Classification-Transfer-Learning-master/
  inflating: /content/dataset/Image-Classification-Transfer-Learning-master/Augmentation.py  
   creating: /content/dataset/Image-Classification-Transfer-Learning-master/Dataset/
   creating: /content/dataset/Image-Classification-Transfer-Learning-master/Dataset/test/
  inflating: /content/dataset/Image-Classification-Transfer-Learning-master/Dataset/test/IMG_0.jpg  
 extracting: /content/dataset/Image-Classification-Transfer-Learning-master/Dataset/test/IMG_1001.jpg  
  inflating: /content/dataset/Image-Classification-Transfer-Learning-master/Dataset/test/IMG_1007.jpg  
  inflating: /content/dataset/Image-Classification-Transfer-Learning-master/Dataset/test/IMG_1011.jpg  
  inflating: /content/dataset/Image-Classification-Transfer-Learning-master/Dataset/test/IMG_1023.jpg  
  inflat

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import os


In [ ]:
train_dir = '/content/dataset/Image-Classification-Transfer-Learning-master/Dataset/training'
test_dir = '/content/dataset/Image-Classification-Transfer-Learning-master/Dataset/test'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical'
)



Found 1554 images belonging to 20 classes.
Found 0 images belonging to 0 classes.


In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create a DataFrame for test images
test_image_files = [f for f in os.listdir(test_dir) if f.endswith('.jpg')]  # Adjust if needed for other formats
test_image_paths = [os.path.join(test_dir, f) for f in test_image_files]

# Create a dataframe with a default label (e.g., 'unknown' class)
test_df = pd.DataFrame({
    'filename': test_image_paths,
    'class': ['unknown'] * len(test_image_paths)  # Placeholder class
})

# Set up test data generator
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='filename',
    y_col=None,  # Since you don't have labels for test images
    target_size=(224, 224),
    batch_size=16,
    class_mode=None,  # No labels needed
    shuffle=False  # Don't shuffle so you can keep track of the order
)


Found 500 validated image filenames.


In [ ]:
# Load the ResNet152V2 model without the top layer
base_model = ResNet152V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
base_model.trainable = False

# Add custom classification layers on top of ResNet152V2
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dense(512, activation='relu'),
    Dense(20, activation='softmax')  # 20 classes
])



In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(train_generator, validation_data=test_generator, epochs=10, verbose=1)

Epoch 1/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.9467 - loss: 0.2198

ValueError: None values not supported.

In [ ]:
base_model.trainable = True

# Recompile the model with a lower learning rate for fine-tuning
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Continue training
fine_tune_history = model.fit(train_generator, validation_data=test_generator, epochs=5, verbose=1)


In [ ]:
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print(f"Test accuracy: {test_acc * 100:.2f}%")


In [ ]:
print(train_generator.image_shape)
print(train_generator.batch_size)


(224, 224, 3)
32


In [ ]:
from tensorflow.keras.applications import ResNet152V2

model = ResNet152V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [ ]:
from tensorflow.keras import layers, models

x = model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
predictions = layers.Dense(20, activation='softmax')(x)  # 20 classes

model = models.Model(inputs=model.input, outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
train_generator.reset()
test_generator.reset()


In [ ]:
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=10,
    verbose=1
)


Epoch 1/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 61s/step - accuracy: 0.3197 - loss: 2.5340  

ValueError: Must provide at least one structure